In [ ]:
from pymatgen.ext.matproj import MPRester
from pymatgen.phonon.bandstructure import PhononBandStructureSymmLine
import numpy as np
from pymatgen.phonon.plotter import PhononBSPlotter
from itertools import product
from scipy.stats import linregress
from pymatgen.core import Lattice
from pymatgen.electronic_structure.bandstructure import Kpoint
from pymatgen.core.structure import Structure
from pymatgen.core.sites import Site


In [ ]:
m = MPRester("903fk7jExqhMBNCB46b")
structure = m.get_structure_by_material_id("mp-862947")
phonon_bs = m.get_phonon_bandstructure_by_material_id("mp-862947")

In [ ]:
#Créer le plot de la structure de bande de phonons et visualiser les branches acoustiques
plotter = PhononBSPlotter(phonon_bs)
plotter.show()

In [ ]:
### Je n'ai pas su résoudre les erreurs de ce code qui selon moi devrait fonctionner ###

phonon_dict = phonon_bs.as_dict()
print(phonon_dict['lattice_rec'])
print(phonon_dict.keys())
# Récupérer les fréquences à partir du dictionnaire phonon_dict
frequencies = phonon_dict['bands'][0]
qpoints = phonon_dict['qpoints']

structure_dict = phonon_dict['structure']
lattice_matrix = structure_dict['lattice']['matrix']
sites = [Site.from_dict(site_dict) for site_dict in structure_dict['sites']]
structure = Structure(lattice_matrix, sites)

lattice_rec = structure.lattice.reciprocal_lattice

kpoints = [Kpoint(q, lattice_rec) for q in qpoints]
kpoints = [kp.as_dict() for kp in kpoints]

directions = [('Gamma', 'X'), ('X', 'W'), ('W', 'K')]
sym_points = ['Gamma', 'X', 'W', 'K']

# Parcourir tous les q-points et trouver les directions recherchées
for d in directions:
    # Trouver les indices des q-points dans cette direction
    indices = [i for i, q in enumerate(qpoints) if (q.label == d[0] and q.label_next == d[1])]
    if len(indices) == 0:
        print(f"Direction {d} not found.")
    else:
        # Extraire les fréquences pour ces q-points
        freqs = np.array([frequencies[i] for i in indices])
        slope = linregress(indices, freqs)
        # Calculer la vitesse du son
        vsound = slope.slope * 2 * np.pi * speed_of_sound / (2 * np.pi / np.linalg.norm(np.array(qpoints[indices[-1]].frac_coords) - np.array(qpoints[indices[0]].frac_coords)))
        print(f"Vitesse du son pour la direction {d}: {vsound:.2f} m/s")
